# 检索器调优方法
+ 上下文压缩
+ 排序
+ 相似性分输
+ 混合搜索

In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)

In [11]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    model="BAAI/bge-m3",
    # https://api.siliconflow.cn/v1/embeddings
    base_url=os.environ.get("SILICONFLOW_API_BASE"),
    api_key=os.environ.get("SILICONFLOW_API_KEY"),
)


## 上下文压缩
+ LLMChainExtractor
+ LLMChainFilter
+ 多个压缩器组合管道

In [3]:
def pretty_print_docs(doc):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i + 1}:\n\n" + d.page_content for i, d in enumerate(doc)]
        )
    )

### 未优化前使用向量库自带能力

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter


documents = TextLoader("../../sources/test.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriver = FAISS.from_documents(texts, embeddings_model).as_retriever()

docs = retriver.invoke("如何进行模型部署")
pretty_print_docs(docs)

Created a chunk of size 1587, which is longer than the specified 1000
Created a chunk of size 1269, which is longer than the specified 1000


Document 1:

痕，是由克劳德造成的，并在事件发生时因巨大冲击而部分失忆。[4]原版策划者之一的加藤 正人提出了一个旨在暗示蒂法和克劳德发生性关系的场景，但被北濑佳范用一个变淡的色调所 取代。􏰀岛一成在接受采访时说，没有一个开发团队人员认为当时的场景会成为一个问题。 [5]
《最终幻想VII补完计划》
2005年，蒂法出现在CG电影《最终幻想VII 降临之子》中，故事发生在原版游戏剧情结束 两年后。在其中，她试图给予克劳德情感上的支持，敦促克劳德放下他对自己施加的不必要的 罪恶感。此外，她还照顾巴雷特的养女玛琳和克劳德在爱丽丝的教堂发⻔口救下的孩子丹泽 尔。在电影中，她与萨菲罗斯的其中一个思念体罗兹战斗，后来她帮助与被召唤的生物巴哈姆 特战斗。编剧􏰀岛一成将她在视频中的⻆色描述为“非常像任何被男人抛弃的女人”，并表示 尽管他们不希望她显得笨拙，但他们也想描绘出从她受到克劳德的情感伤害出发。[6]在视频 的初稿中，她原本计划在当时的短片中扮演更􏰁要的⻆色，该短片仅以蒂法，克劳德和几个孩 子为主。
蒂法也在前传游戏《最终幻想VII 危机之前》和《最终幻想VII 核心危机》以及OVA《最终 幻想VII 最终命令》中登场。每次登场时，她的出现都与尼布尔海姆的毁灭有关。[2]官方 小说《通向微笑之路》中有专⻔的《蒂法篇》，讲述了原版游戏和降临之子两段之间的故事。 从蒂法的⻆度出发，详细讲述了她如何在Edge City创建一个新的第七天堂酒吧，并试图坚 持自己和克劳德的正常家庭观念，尽管克劳德逐渐开始逃避与他人接触。蒂法还短暂出现在游 戏《最终幻想VII 地狱犬的挽歌》中，该游戏的剧情在降临之子故事一年后发生，她在游戏 中帮助主⻆文森特·瓦伦丁捍卫星球，抵抗怪物欧米茄和“兵器”。她后来出现在游戏的结尾 中，讨论着文森特的失踪。[2]
其他登场 在《最终幻想VII补完计划》之外，蒂法还出现在格斗游戏《神佑擂台》中，作为可解锁的⻆ 色和可选的Boss。[7]她后来出现在电子棋盘游戏《富豪街》中。在《王国之心II》中，她 穿着自己在降临之子中的服装，寻找克劳德，然后与该系列的怪物“无心”战斗。她原本计划 出现在原版《王国之心》的最终合辑中，但由于时间限制，工作人员选择改用萨菲罗斯。[8] 蒂法也是格斗游戏《最终幻想 纷争012》中的玩家⻆色之一，该游戏的⻆色来自各种《最终 

### 使用LLMChainExtractor压缩器
基础检索器ContextualCompressionRetriever以及LLMChainExtractor，它将迭代最初返回的文档，并从每个文档种仅提取与查询相关的内容

In [6]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)  # 基础的压缩器
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriver
)

compressed_docs = compression_retriever.invoke("如何进行模型部署")
pretty_print_docs(compressed_docs)

### LLMChainFilter：使用LLM链来决定过滤掉哪些最初检索到的文档以及返回哪些文档，而无需操作文档的内容

In [7]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriver
)

compressed_docs = compression_retriever.invoke("如何进行模型部署")
pretty_print_docs(compressed_docs)

### 多个压缩器组合管道
+ 使用DocumentCompressorPipeline轻松地按顺序组合多个压缩器
+ 将文档拆解成更小的碎片
+ 删除冗余文档
+ 串联多个压缩器

In [ ]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter
from langchain.retrievers.document_compressors import EmbeddingsFilter

splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings_model)
relevant_filter = EmbeddingsFilter(
    embeddings=embeddings_model, similarity_threshold=0.76
)
# 串联起来
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [10]:
from langchain.retrievers import ContextualCompressionRetriever

compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriver
)

compressed_docs = compression_retriever.invoke("如何进行模型部署")
pretty_print_docs(compressed_docs)

## 排序

In [13]:
from langchain_core.vectorstores import InMemoryVectorStore

texts = [
    "西湖是杭州著名的旅游景点。",
    "我最喜欢的歌曲是《月亮代表我的心》。",
    "故宫是北京最著名的古迹之一。",
    "这是一篇关于北京故宫历史的文档。",
    "我非常喜欢去电影院看电影。",
    "北京故宫的藏品数量超过一百万件。",
    "这只是一段随机文本。",
    "《三国演义》是中国四大名著之一。",
    "紫禁城是故宫的别称，位于北京。",
    "故宫博物院每年接待游客数百万人次。",
]

# 创建检索器
retriever = InMemoryVectorStore.from_texts(
    texts, embedding=embeddings_model
).as_retriever(search_kwargs={"k": 10})
query = "请告诉我关于故宫的信息?"

# 获取按相关性排序的文档
docs = retriever.invoke(query)
for doc in docs:
    print(f"- {doc.page_content}")


- 这是一篇关于北京故宫历史的文档。
- 故宫是北京最著名的古迹之一。
- 这只是一段随机文本。
- 故宫博物院每年接待游客数百万人次。
- 我最喜欢的歌曲是《月亮代表我的心》。
- 西湖是杭州著名的旅游景点。
- 北京故宫的藏品数量超过一百万件。
- 紫禁城是故宫的别称，位于北京。
- 《三国演义》是中国四大名著之一。
- 我非常喜欢去电影院看电影。


请注意，返回的文档按与查询的相关性降序排列。LongContextReorder文档转换器将实现上述重新排序

In [14]:
from langchain_community.document_transformers import LongContextReorder

# 重新排序文档:
# 相关性较低的文档将位于列表中间
# 相关性较高的文档将位于开头和结尾
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# 确认相关性高的文档位于开头和结尾
for doc in reordered_docs:
    print(f"- {doc.page_content}")

- 故宫是北京最著名的古迹之一。
- 故宫博物院每年接待游客数百万人次。
- 西湖是杭州著名的旅游景点。
- 紫禁城是故宫的别称，位于北京。
- 我非常喜欢去电影院看电影。
- 《三国演义》是中国四大名著之一。
- 北京故宫的藏品数量超过一百万件。
- 我最喜欢的歌曲是《月亮代表我的心》。
- 这只是一段随机文本。
- 这是一篇关于北京故宫历史的文档。


In [15]:
# 整合到链中

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate

prompt_template = """
Given these texts:
----
{context}
----
Please answer the following question:
{query}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "query"])

# Create and invoke the chain:
chain = create_stuff_documents_chain(llm, prompt)
response = chain.invoke({"context": reordered_docs, "query": query})
print(response)

故宫，又称紫禁城，是北京最著名的古迹之一，位于北京市中心。作为明清两代的皇家宫殿，故宫不仅是中国古代建筑艺术的杰作，也是世界文化遗产之一。以下是关于故宫的一些详细信息：

1. **历史地位**：故宫始建于明永乐四年（1406年），是明清两代24位皇帝的皇宫，具有近600年的历史。

2. **规模与藏品**：故宫博物院现藏有超过一百万件文物，涵盖绘画、陶瓷、青铜器、玉器等各类珍贵文化遗产，是世界上保存最完整、规模最大的木质结构古建筑群之一。

3. **旅游热度**：故宫每年吸引数百万游客参观，是中国最受欢迎的旅游景点之一。

4. **建筑特色**：故宫占地约72万平方米，拥有980余座建筑，以中轴线对称布局，体现了中国传统宫殿建筑的庄严与恢弘。

5. **文化意义**：作为中华文明的象征，故宫不仅展示了古代皇权制度，还承载了丰富的历史文化内涵。

如需更深入的历史或专题信息（如建筑细节、特定文物等），可进一步说明。


## 相似性分数
+ 根据相似性打分过滤
+ 为文档添加分数

### 相似性分数搜索

In [ ]:
# 初始化Chroma客户端

from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings_model,
    # persist_directory="chroma_langchain_db",  # 可选参数，指定持久化目录
)

In [25]:
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

# 添加文档内容到向量数据库种
vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings_model,
)

ConnectTimeout: _ssl.c:993: The handshake operation timed out in add.

In [23]:
# 添加一组文档
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)
document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)
document_3 = Document(
    page_content="Building an exciting new project with Langchain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)
document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)
document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)
document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)
document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)
document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)
document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)
document_10 = Document(
    page_content="I hgve a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)
documents = [
    document_1,
    document_3,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

uuids = [str(uuid4()) for _ in range(len(documents))]

# 存储到向量数据库中
vector_store.add_documents(documents=documents, ids=uuids)

['910b9cbf-6e35-4ecc-9910-7bfff1d1e3d4',
 'f03435eb-492a-4298-852f-604487fe37de',
 'e4acd253-0faa-4c2e-92e0-89ec01eb1358',
 '34f73d01-2dfd-4069-ad62-f7912ae94266',
 'f1b24fe0-b720-424d-85c0-9b432271f97a',
 'fbc5e7d9-74fd-4279-9a7f-491c1bc6bc38',
 'f944077a-9466-4b0b-99d5-f298bc11763a',
 '223567e1-8d61-4855-a79b-8c97138e0382',
 '9c636561-a8d6-480d-9dbd-bd5c1ea181e0',
 '4b968e97-ffa5-47a0-bcd6-035801fb1b2f']

In [22]:
# 使用相似性分数检索
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "nesw"}
)

for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

### 为文档添加分数
通过一个自定义链，可以为原始文档增加相关性评分

In [26]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vector_store.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["'score"] = score
    return docs

In [27]:
retriever.invoke("Robbers")

(Document(id='ed7e9c50-4f02-4e65-8f18-f2432aae25d1', metadata={'source': 'tweet', "'score": 0.7793059349060059}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(id='910b9cbf-6e35-4ecc-9910-7bfff1d1e3d4', metadata={'source': 'tweet', "'score": 0.7793059349060059}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(id='ea97f262-4a9b-444f-9949-1515925c66cf', metadata={'source': 'news', "'score": 0.9148495197296143}, page_content='The stock market is down 500 points today due to fears of a recession.'),
 Document(id='9c636561-a8d6-480d-9dbd-bd5c1ea181e0', metadata={'source': 'news', "'score": 0.9148495197296143}, page_content='The stock market is down 500 points today due to fears of a recession.'))